In [21]:
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, generate_molecule

N = 8000
n_sample = 1000
lam = 30
nqubit = 4

molecule = generate_molecule("H", "H", 0.7414, "sto-3g")


converged SCF energy = -1.11668438708534


## Hamiltonian

In [22]:
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule)

Starting to parse FermionOperator using 4 qubits...

Operator t:  -0.09886396933545816 [] +
0.04532220205287399 [X0 Z1 X2] +
0.04532220205287399 [X0 Z1 X2 Z3] +
0.04532220205287399 [Y0 Z1 Y2] +
0.04532220205287399 [Y0 Z1 Y2 Z3] +
0.17119774903432977 [Z0] +
0.17119774903432972 [Z0 Z1] +
0.165867024105892 [Z0 Z1 Z2] +
0.165867024105892 [Z0 Z1 Z2 Z3] +
0.12054482205301804 [Z0 Z2] +
0.12054482205301804 [Z0 Z2 Z3] +
0.1686221915892094 [Z1] +
-0.2227859304041847 [Z1 Z2 Z3] +
0.17434844185575682 [Z1 Z3] +
-0.22278593040418473 [Z2]
Term, coeff:  () -0.09886396933545816
Term, coeff:  ((0, 'Z'),) 0.17119774903432977
Index, p_char:  0 Z
Term, coeff:  ((0, 'Z'), (1, 'Z')) 0.17119774903432972
Index, p_char:  0 Z
Index, p_char:  1 Z
Term, coeff:  ((2, 'Z'),) -0.22278593040418473
Index, p_char:  2 Z
Term, coeff:  ((1, 'Z'), (2, 'Z'), (3, 'Z')) -0.2227859304041847
Index, p_char:  1 Z
Index, p_char:  2 Z
Index, p_char:  3 Z
Term, coeff:  ((1, 'Z'),) 0.1686221915892094
Index, p_char:  1 Z
Term, coeff:  

## UCCSD

In [23]:
uccsd = UCCSD(4, molecule)
paulis = uccsd.paulis
num_operators = len(paulis)

In [24]:
import random, logging

from qwrapper.obs import PauliObservable
from qwrapper.hamiltonian import HeisenbergModel, compute_ground_state
from qwrapper.optimizer import AdamOptimizer, UnitLRScheduler, PrintMonitor, FileMonitor
from qswift.initializer import ZBasisInitializer
from gqe.simple_model.model import SimpleModel, Ansatz
from gqe.energy_estimator.iid import IIDEstimator

INPUT_FILENAME = '../saved_models/model_h2_sto3g.json'
OUTPUT_FILENAME = '../saved_models/model_h2_sto3g_.json'

print("ground state:", compute_ground_state(hamiltonian))
random.seed(23)
ansatz = Ansatz([random.gauss(0, 1) for _ in range(num_operators)],
                paulis, nqubit=nqubit)
estimator = IIDEstimator(hamiltonian,
                         ZBasisInitializer(),
                         N, K=0, tool='qulacs', n_sample=n_sample, n_grad_sample=1)
model = SimpleModel(estimator, ansatz, N, lam, n_sample)
monitors = [PrintMonitor(), FileMonitor('../output/energy_h2_sto3g.txt')]
model.run(AdamOptimizer(maxiter=400, scheduler=UnitLRScheduler(0.01), monitors=monitors))
for m in monitors:
    m.finalize()
with open(OUTPUT_FILENAME, 'w') as f:
    f.write(model.ansatz.toJSON())

ground state: -1.038406205325445
0 0.4895727073344513
1 0.4641038754869249
2 0.43992683293563795
3 0.4161898639790946
4 0.3918534304894865
5 0.36674338795325606
6 0.3409221402680764
7 0.31416737267041983
8 0.2867845810233016
9 0.2593644484356889
10 0.23160361181034803
11 0.2027478194307241
12 0.1729741154225354
13 0.1437828470955465
14 0.11458329084837315
15 0.08584954427811109
16 0.05670937563017009
17 0.02733717146879136
18 -0.0026781943868830482
19 -0.032792479681209186
20 -0.06291540572410298
21 -0.09200405055917601
22 -0.1200988042373288
23 -0.14741968632704086
24 -0.17406972865757847
25 -0.20080165152673854
26 -0.22618677290395225
27 -0.2497095501325118
28 -0.2724394070607657
29 -0.293692311448876
30 -0.3137133173671374
31 -0.33215191634989655
32 -0.34898231612186853
33 -0.3641834075557163
34 -0.37787905445929065
35 -0.3896084228106841
36 -0.3999225756432379
37 -0.4089778566939301
38 -0.4168640410734261
39 -0.42370015565832486
40 -0.4295750467266858
41 -0.4346041676210117
42 -0.4